In [62]:
import shapely
from shapely.geometry import Point, Polygon, LineString, GeometryCollection

In [63]:
class polygon:
    def __init__(self,layer,datatype,noOfpoints):
        self.layer = layer
        self.datatype = datatype
        self.noOfpoints = noOfpoints
        self.points = []

    def addPoints(self,points):
        self.points.append(points.split(" "))

### reading the source txt file

In [64]:
source = []
with open('./MileStone_Input/Milestone 2/Source.txt','r') as file:
    for line in file:
        source.append(line)

i = 0

source_header = []
source_polygons = []
source_footer = []

while i<len(source):
    if "header" in source[i]:
        while source[i] != "boundary\n":
            source_header.append(source[i])
            i+=1
    
    elif "boundary" in source[i]:
        p1 = polygon(source[i+1].split(" ")[1],source[i+2].split(" ")[1],source[i+3].split("  ")[1])
        for p in range(2,2+int(p1.noOfpoints)):
            p1.addPoints(source[i+3].split("  ")[p])
        source_polygons.append(p1)
        i+=4
    elif 'endstr' in source[i] or 'endlib' in source[i]:
        source_footer.append(source[i])
        i+=1
    else:
        i+=1

print(len(source_polygons))

1007


### Reading the polygon of interest

In [65]:
poi = []
with open('./MileStone_Input/Milestone 2/POI.txt','r') as file:
    for line in file:
        poi.append(line)

i = 0

poi_header = []
poi_polygons = []
poi_footer = []

while i<len(poi):
    if "header" in poi[i]:
        while poi[i] != "boundary\n":
            poi_header.append(poi[i])
            i+=1
    
    elif "boundary" in poi[i]:
        p1 = polygon(poi[i+1].split(" ")[1],poi[i+2].split(" ")[1],poi[i+3].split("  ")[1])
        for p in range(2,2+int(p1.noOfpoints)):
            p1.addPoints(poi[i+3].split("  ")[p])
        poi_polygons.append(p1)
        i+=4
    elif 'endstr' in poi[i] or 'endlib' in poi[i]:
        poi_footer.append(poi[i])
        i+=1
    else:
        i+=1

print(poi_header)
print(poi_polygons)
print(poi_footer)

['header 600 \n', 'bgnlib 1/17/2023 18:06:45 1/17/2023 18:06:45 \n', 'libname drc.db\n', 'units 0.0001 1e-10 \n', '\n', 'bgnstr 1/17/2023 18:06:45 1/17/2023 18:06:45 \n', 'strname top\n', '\n']
['endstr\n', 'endlib\n']


### gathering info about poi using shapely

In [66]:
int_points = []
for point in poi_polygons[0].points:
    temp = [int(point[0]),int(point[1])]
    int_points.append(temp)

poi_poly = Polygon(int_points)

### Find the polygons in source text

In [67]:
found_polygons = []

for i in range(0,len(source_polygons)):
    int_points = []
    for point in source_polygons[i].points:
        temp = [int(point[0]),int(point[1])]
        int_points.append(temp)

    source_poly = Polygon(int_points)

    if poi_poly.area == source_poly.area:
        found_polygons.append(source_polygons[i])
    

print(len(found_polygons))


1000


### Writing header lines

In [68]:
file1 = open("milestone2.txt","w")
 
file1.writelines(source_header)
file1.close()

### Writing polygons

In [69]:
output_polygons = []
for i in range(0,len(found_polygons)):
    output_polygons.append("boundary\n")
    output_polygons.append("layer "+found_polygons[i].layer)
    output_polygons.append("datatype "+found_polygons[i].datatype)
    output_polygons.append("xy "+found_polygons[i].noOfpoints)
    for point in found_polygons[i].points:
        output_polygons.append("  "+point[0]+" "+point[1])
    output_polygons.append('endel\n')

file1 = open("milestone2.txt","a")

file1.writelines(output_polygons)
file1.close()

In [70]:
file1 = open("milestone2.txt","a")

file1.writelines(source_footer)
file1.close()